<a href="https://colab.research.google.com/github/Storm00212/QUANT/blob/main/Cairo_ALGO_vs_S%26P_500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance scipy


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
data = yf.download("SPY", start="2020-01-01", end="2024-01-01")
prices = data["Adj Close"]

# Log returns
returns = np.log(prices / prices.shift(1)).dropna()


In [ ]:
vol = returns.rolling(20).std()
signal = returns.rolling(10).mean()

# Risk-normalized weights
weights = signal / vol
weights = weights.clip(-2, 2)

strategy_returns = weights.shift(1) * returns
strategy_returns.dropna(inplace=True)


In [ ]:
window = 60
bins = np.linspace(-0.05, 0.05, 80)

X, Y, Z = [], [], []

for i in range(window, len(strategy_returns)):
    window_data = strategy_returns[i-window:i]
    kde = gaussian_kde(window_data)
    density = kde(bins)

    X.append(np.full_like(bins, i))
    Y.append(bins)
    Z.append(density)

X = np.array(X)
Y = np.array(Y)
Z = np.array(Z)


In [ ]:
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection="3d")

ax.plot_surface(
    X,
    Y,
    Z,
    cmap="viridis",
    linewidth=0,
    antialiased=True
)

ax.set_xlabel("Time")
ax.set_ylabel("Returns")
ax.set_zlabel("Density")
ax.set_title("3D Return Distribution Evolution")

plt.show()


In [ ]:
capital = 100_000

strategy_equity = capital * np.exp(strategy_returns.cumsum())
buy_hold_equity = capital * np.exp(returns.loc[strategy_returns.index].cumsum())

plt.figure(figsize=(12,6))
plt.plot(strategy_equity, label="Strategy", color="green")
plt.plot(buy_hold_equity, label="S&P 500", linestyle="--", color="red")

plt.title("Equity Curve Comparison")
plt.ylabel("Portfolio Value ($)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
def stats(r):
    return {
        "CAGR": np.exp(r.mean()*252)-1,
        "Volatility": r.std()*np.sqrt(252),
        "Sharpe": r.mean()/r.std()*np.sqrt(252),
        "Skewness": r.skew(),
        "Kurtosis": r.kurtosis()
    }

stats(strategy_returns)
